# <ins>Evaluation of Generator Reactive Power Capability Characteristics in Load Flow Analysis</ins>

By incorporating the reactive power capability characteristics of a generator, the minimum **(Qmin)** and maximum **(Qmax)** reactive power limits can be determined as a function of the generator’s active power output. This directly influences the load flow result, which will be demonstrated in subsequent sections of this tutorial.

The **q_capability_curve_table** provides a structured representation of the generator’s reactive power limits and consists of the following key variables:
  * **id_q_capability_curve** – Identifier for the reactive power capability curve in the **q_capability_curve_table**.  <br>
  * **min_q_mvar** – Minimum reactive power limit (Qmin). <br>
  * **max_q_mvar** – Maximum reactive power limit (Qmax). <br>

To facilitate the integration of these limits, the function **create_q_capability_characteristics_object** is employed. This function transforms the provided reactive power curve data into a characteristic object, applying linear interpolation to construct a continuous function. The resulting characteristic is stored in the **q_capability_characteristic** data structure.

Following this, a diagnostic function is executed to identify and report any inconsistencies within the dataset, ensuring data integrity.

In [ ]:
import numpy as np
import pandas as pd

from pandapower.control.util.auxiliary import create_q_capability_characteristics_object, plot_characteristic
from pandapower.control.util.diagnostic import q_capability_curve_table_diagnostic
from pandapower.run import runpp
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_ext_grid,
    create_line_from_parameters,
    create_load,
    create_transformer_from_parameters,
    create_gen
)

In [ ]:
#Create network
def test_net_for_q_capability_curve(characteristics_data=False):
    net = create_empty_network()
    bus1 = create_bus(net, name="bus1", vn_kv=20., type="b", min_vm_pu=0., max_vm_pu=1.05)
    bus2 = create_bus(net, name="bus2", vn_kv=110., type="b", min_vm_pu=0., max_vm_pu=1.05)
    bus3 = create_bus(net, name="bus3", vn_kv=110., type="b", min_vm_pu=0., max_vm_pu=1.05)

    create_ext_grid(net, bus3, name="External Grid", vm_pu=1.0, va_degree=0.0,max_p_mw=100000, min_p_mw=0, min_q_mvar=-300, max_q_mvar=300,
                       s_sc_max_mva=10000, s_sc_min_mva=8000, rx_max=0.1, rx_min=0.1)
    # create lines
    create_line_from_parameters(net, bus2, bus3, length_km=10,df=1,max_loading_percent=100,vn_kv=110,max_i_ka=0.74,type="ol",
                   r_ohm_per_km=0.0949, x_ohm_per_km =0.38, c_nf_per_km=0.0092, name="Line")
    # create load
    create_load(net, bus3, p_mw=200, q_mvar=180, name="Load", vm_pu=1.0 )

    # create transformer
    create_transformer_from_parameters(net, bus2, bus1, name="110kV/20kV transformer", parallel=1,max_loading_percent=100,sn_mva=210,
    vn_hv_kv=110, vn_lv_kv=20, vk_percent=12.5, vkr_percent=0.01904762, vk0_percent=10, vkr0_percent=0,
                          shift_degree=330, vector_group="YNd11", i0_percent= 0.26, pfe_kw=0,si0_hv_partial=0.5)

    create_gen(net, bus1, p_mw=100, sn_mva=255.0, scaling=1.0, type="Hydro",
                 cos_phi=0.8, pg_percent=0.0, vn_kv=19.0, vm_pu=1.0)
    if characteristics_data:
        net["q_capability_curve_table"] = pd.DataFrame(
        {'id_q_capability_curve': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'p_mw':      [-331.01001, -298.0,    -198.0,     -66.2000,   -0.1,        0,           0.1,        66.200, 100,     198.00,     298.00,     331.01001],
        'q_min_mvar':[-0.0100,    -134.0099, -265.01001, -323.01001, -323.01001,  -323.01001, -323.01001, -323.01001, 0, -265.01001, -134.00999, -0.01000 ],
        'q_max_mvar':[0.01000,    134.00999,  228.00999,  257.01001,  261.01001,  261.01001,   261.01001,  257.01001, 30,  40,  134.0099,   0.01]})

        net.gen.loc[0,"id_q_capability_characteristic"] = 0
        net.gen['curve_style'] = "straightLineYValues"

    return net

#### <ins>No reactive power limit applied<ins>

In [ ]:
net1 = test_net_for_q_capability_curve()
runpp(net1)
net1.gen.loc[0]

Upon examining the lookup table, it is observed that no values were initially assigned to the variables **min_q_mvar** and **max_q_mvar**. These values are now assigned, thereby completing the definition of the generator’s reactive power characteristic.

#### <ins>General reactive power limits enforced</ins>

In [ ]:
net2 = test_net_for_q_capability_curve()
net2.gen.loc[0,"max_q_mvar"] = 50.0
net2.gen.loc[0, "min_q_mvar"] =  -3

In [ ]:
runpp(net2, enforce_q_lims=True)
assert net2.res_gen.q_mvar.loc[0] == -3
assert net2.res_gen.p_mw.loc[0] == 100

In [ ]:
net2.gen.loc[0]

#### <ins>Create Q characteristics table</ins>

In [ ]:
net3 = test_net_for_q_capability_curve(characteristics_data=True)

In [ ]:
# Add q_capability_characteristic for one gen based on q_capability_curve_table
create_q_capability_characteristics_object(net3)

The generator's characteristic has now been added to the dataframe. Refer to the lookup table **q_capability_curve_table** below for details:

In [ ]:
net3.q_capability_curve_table

After executing the **create_q_capability_characteristics_object** function, the following column in the **net.gen** are updated:  **reactive_capability_curve**

In [ ]:
net3.gen.loc[0]

The **q_capability_characteristic** table is look like below:

In [ ]:
net3.q_capability_characteristic

The characteristic function created within this process is a **Characteristic** object, utilizing SciPy’s **interp1d** for **linear interpolation**.

In [ ]:
plot_characteristic(net3.q_capability_characteristic.q_max_characteristic.at[0], -331, 331,
                               xlabel='Values of "P_mw"', ylabel='Value of "Q_max_Mvar"')

In [ ]:
plot_characteristic(net3.q_capability_characteristic.q_min_characteristic.at[0], -331, 331,
                               xlabel='Values of "P_mw"', ylabel='Value of "Q_min_Mvar"')

In [ ]:
runpp(net3, enforce_q_lims=True)
assert net3.res_gen.q_mvar.loc[0] == 0
assert net3.res_gen.p_mw.loc[0] == 100

# Validation and Impact on Load Flow
To verify the impact of the adjusted values on the power system simulation:
* Voltage magnitudes at generator buses are examined. <br>
* Reactive power generation is observed to remain constant when the generator reaches its defined limits during load flow analysis.
* A comparative analysis is performed to evaluate the difference in results across three cases:
    * No reactive power limit applied <br>
    * General reactive power limits enforced<br>
    * Limits derived from the characteristic table

In [ ]:
print("\n --------No reactive power limit applied-----------------\n")
print(f"Qmin={net1.gen.min_q_mvar.loc[0]}")
print(f"QMax={net1.gen.max_q_mvar.loc[0]}\n")
print(net1.res_gen)
print(net1.res_bus)
print("\n --------General reactive power limits enforced-------------\n")
print(f"Qmin={net2.gen.min_q_mvar.loc[0]}")
print(f"QMax={net2.gen.max_q_mvar.loc[0]}\n")
print(net2.res_gen)
print(net2.res_bus)
print("\n --------Limits derived from the characteristic table---------\n")
print(f"Qmin={net3.gen.min_q_mvar.loc[0]}")
print(f"QMax={net3.gen.max_q_mvar.loc[0]}\n")
print(net3.res_gen)
print(net3.res_bus)

The results confirm that applying the reactive power capability characteristics leads to distinct voltage profiles and a controlled reactive power response, significantly influencing the overall power system behavior.

# Diagnostic
The
diagnostic
function
for generator reactive power characteristics checks the following:

* Are there Q capability curve characteristic table?
* Are there ** reactive_capability_curve **, ** curve_style ** and ** id_q_capability_characteristic ** missing in net.gen or net.sgen?
*Check if all
relevant
columns
for creating Q capability characteristics are populated in the ** q_capability_curve_table **
*Check
the
data
types
of ** reactive_capability_curve ** and ** id_q_capability_characteristic **
* Are there any missing ** id_q_capability_characteristic ** values in the ** q_capability_curve_table ** ?

The
results
are
displayed
to
the
user
with the help of warning statements.

In [ ]:
net4 = test_net_for_q_capability_curve(characteristics_data=True)
create_q_capability_characteristics_object(net4)
q_capability_curve_table_diagnostic(net4, "gen")
#runpp(net4, enforce_q_lims=True)

In [ ]:
net4.gen.loc[0,"reactive_capability_curve"] = False
q_capability_curve_table_diagnostic(net4, "gen")

In [ ]:
net4 = test_net_for_q_capability_curve(characteristics_data=True)
create_q_capability_characteristics_object(net4)
net4.gen.loc[0,"reactive_capability_curve"] = False
net4.gen.loc[0,"id_q_capability_characteristic"] = np.nan
q_capability_curve_table_diagnostic(net4, "gen")

In [ ]:
net4 = test_net_for_q_capability_curve(characteristics_data=True)
create_q_capability_characteristics_object(net4)
net4.gen.loc[0, "curve_style"] = "hi"
net4.q_capability_curve_table.drop(columns="q_min_mvar")
q_capability_curve_table_diagnostic(net4, "gen")

In [ ]:
net4 = test_net_for_q_capability_curve(characteristics_data=True)
create_q_capability_characteristics_object(net4)
net4.q_capability_curve_table.q_min_mvar = np.nan
net4.q_capability_curve_table.drop(columns="q_min_mvar")
q_capability_curve_table_diagnostic(net4, "gen")